# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [22]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from skimage.transform import resize #Library to resize images to a target size.

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [15]:
train_doc = np.random.permutation(open('C:\\Users\\varmag\\Downloads\\Project_data\\Project_data\\train.csv').readlines())
val_doc = np.random.permutation(open('C:\\Users\\varmag\\Downloads\\Project_data\\Project_data\\val.csv').readlines())
batch_size = 10

In [14]:
train_doc = train_doc[:50]
val_doc = val_doc[:10]

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [52]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.arange(0,20)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 5
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,20,250,250,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #code to resize the images
                    image = imresize(image, (250,250))
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0])
                    batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1])
                    batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            
        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [34]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:\\Users\\varmag\\Downloads\\Project_data\\Project_data\\train'
val_path = 'C:\\Users\\varmag\\Downloads\\Project_data\\Project_data\\val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 2
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 2


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [69]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

input_shape = (20,250, 250, 3)
num_classes = 5
#Conv3D model architecture
conv3D_model = Sequential()

conv3D_model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu',
                 input_shape=input_shape))

conv3D_model.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))               
conv3D_model.add(MaxPooling3D(pool_size=(2, 2, 2)))
conv3D_model.add(Dropout(0.25))

# flatten and put a fully connected layer
conv3D_model.add(Flatten())
conv3D_model.add(Dense(128, activation='relu')) # fully connected
conv3D_model.add(Dropout(0.5))
                 
conv3D_model.add(Dense(num_classes, activation='softmax'))

conv3D_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 18, 248, 248, 32)  2624      
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 16, 246, 246, 64)  55360     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 8, 123, 123, 64)   0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 8, 123, 123, 64)   0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 7746048)           0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               991494272 
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

In [70]:
conv3D_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [63]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [64]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

SyntaxError: invalid syntax (<ipython-input-64-12a2e9c79c93>, line 10)

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [66]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [71]:
conv3D_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                     validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/2


C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


ResourceExhaustedError: OOM when allocating tensor with shape[7746048,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node dense_4/random_uniform/RandomUniform}} = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=30, seed2=7416729, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_4/random_uniform/RandomUniform', defined at:
  File "C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\varmag\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Us